In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# カレントスレッドにデフォルトのグラフが残存していることがあるので（特にJupyter Notebookを使っている場合）、リセットしておく
tf.reset_default_graph()
sess = tf.InteractiveSession()

# 再現性の確保のために乱数シードを固定（数値は何でもよい）
tf.set_random_seed(12345)

# 入力データ
# MNISTのワンホット表現での読み込み
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:
# ドロップアウト付きの全結合
def matmul_plus_bias_with_dropout(x, w, b, p):
  return tf.matmul(tf.nn.dropout(x, keep_prob=p), w) + b


# 0 入力画像
x = tf.placeholder(tf.float32, name='x')

# 1 サイズ変更
x_1 = tf.reshape(x, [-1, 28, 28, 1])

# 2 畳み込み
# ランダムカーネル
k_0 = tf.Variable(tf.truncated_normal([4, 4, 1, 10], mean=0.0, stddev=0.1))
# 畳み込み
x_2 = tf.nn.conv2d(x_1, k_0, strides=[1, 3, 3, 1], padding='VALID')

# 3 活性化関数
x_3 = tf.nn.relu(x_2)

# 4 プーリング
x_4 = tf.nn.max_pool(x_3, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')

# 5 サイズ変更
x_5 = tf.reshape(x_4, [-1, 160])

# 6 全結合
# 重みとバイアス
w_1 = tf.Variable(tf.zeros([160, 40]))
b_1 = tf.Variable([0.1] * 40)
# ドロップアウト率
p_1 = tf.placeholder(1.0, name='p_1')
# 全結合
x_6 = matmul_plus_bias_with_dropout(x_5, w_1, b_1, p_1)

# 7 活性化関数
x_7 = tf.nn.relu(x_6)

# 8 全結合
# 重みとバイアス
w_2 = tf.Variable(tf.zeros([40, 10]))
b_2 = tf.Variable([0.1] * 10)
# ドロップアウト率
p_2 = tf.placeholder(1.0, name='p_2')
# 全結合
x_8 = matmul_plus_bias_with_dropout(x_7, w_2, b_2, p_2)

# 9 確率化
y = tf.nn.softmax(x_8)

# 10 損失関数の最小化
# 正解ラベル
labels = tf.placeholder(tf.float32, name='labels')
# 損失関数（交差エントロピー）と最適化処理（Adam）
loss = -tf.reduce_sum(labels * tf.log(y))
optimizer = tf.train.AdamOptimizer().minimize(loss)

# 11 精度検証
prediction_match = tf.equal(tf.argmax(y, axis=1), tf.argmax(labels, axis=1))
accuracy = tf.reduce_mean(tf.cast(prediction_match, tf.float32), name='accuracy')

# パラメーター
# バッチサイズ
BATCH_SIZE = 32
# 学習回数
NUM_TRAIN = 10_000
# 学習中の出力頻度
OUTPUT_BY = 500
# ドロップアウト率
DROPOUT_PROB_1 = 0.2
DROPOUT_PROB_2 = 0.5


# 学習の実行
sess.run(tf.global_variables_initializer())
dropout_prob = {p_1: DROPOUT_PROB_1, p_2: DROPOUT_PROB_2}

saver = tf.train.Saver()

for i in range(NUM_TRAIN):
  batch = mnist.train.next_batch(BATCH_SIZE)
  inout = {x: batch[0], labels: batch[1]}
  if i % OUTPUT_BY == 0:
    train_accuracy = accuracy.eval(feed_dict={**inout, p_1: 1.0, p_2: 1.0})
    print('step {:d}, accuracy {:.2f}'.format(i, train_accuracy))
    # 過程の保存
    saver.save(sess, 'models/my-model', global_step=i)
  optimizer.run(feed_dict={**inout, **dropout_prob})

# 最終結果の保存
saver.save(sess, 'models/my-model')




step 0, accuracy 0.16
step 500, accuracy 0.41
step 1000, accuracy 0.69
step 1500, accuracy 0.84
step 2000, accuracy 0.75
step 2500, accuracy 0.81
step 3000, accuracy 0.66
step 3500, accuracy 0.84
step 4000, accuracy 0.78
step 4500, accuracy 0.66
step 5000, accuracy 0.91


参考URL https://deepinsider.jp/tutor/introtensorflow/buildcnn